In [13]:
from pathlib import Path
from genetic_algorithm import genetic_algorithm
from genetic_algorithm.selection import TournamentSelection
from mwcnf_parser import parse_mwcnf
from solver import MwcnfGenerator
from solver.fitness import satisfied_clause_count, weights
from solver.mwcnf_individual import MwcnfIndividual

from io import StringIO
import matplotlib.pyplot as plt


In [14]:
from dataclasses import dataclass
from math import ceil, floor


actual_max_weights = {
    "wruf36-157-1.mwcnf": 9151,
    "wruf36-157-2.mwcnf": 5348,
    "wruf36-157-3.mwcnf": 16,
    "wruf36-157-4.mwcnf": 2074,
    "wruf36-157-5.mwcnf": 13604,
    "wruf36-157-6.mwcnf": 5895,
    "wruf36-157-7.mwcnf": 3154,
    "wruf36-157-8.mwcnf": 2088,
    "wruf36-157-9.mwcnf": 3420,
    "wruf36-157-10.mwcnf": 9865,
    "wuf50-01.mwcnf": 4190,
    "wuf50-02.mwcnf": 1469,
    "wuf50-03.mwcnf": 5305,
    "wuf50-04.mwcnf": 1488,
    "wuf50-05.mwcnf": 11315,
    "wuf50-06.mwcnf": 1012,
    "wuf50-07.mwcnf": 9570,
    "wuf50-08.mwcnf": 555,
    "wuf50-09.mwcnf": 1612,
    "wuf50-010.mwcnf": 3031,
    "wuf75-01.mwcnf": 6293,
    "wuf75-02.mwcnf": 6003,
    "wuf75-03.mwcnf": 297,
    "wuf75-04.mwcnf": 1019,
    "wuf75-05.mwcnf": 1950,
    "wuf75-06.mwcnf": 7470,
    "wuf75-07.mwcnf": 5891,
    "wuf75-08.mwcnf": 2747,
    "wuf75-09.mwcnf": 9285,
    "wuf75-010.mwcnf": 6976,
}


@dataclass
class Result:
    formula_name: str
    run_no: int
    clause_cnt: int
    sat_clause_cnt: int
    weight: int
    last_improvement: int
    pop_size: int
    max_weight: int

    def solved(self):
        return self.clause_cnt == self.sat_clause_cnt

    def get_int(self):
        return self.solved() * self.weight / self.max_weight

    def get_penalized_last_improvement(self):
        return (10 - (self.solved() * 9)) * self.last_improvement

stats = {}
with open("out/stats.csv", "rt") as stats_f:
    for line in stats_f.readlines():
        formula_name, run_no, clause_cnt, sat_clause_cnt, weight, last_improvement, pop_size = line.split(sep=",")

        dict_key = f"{clause_cnt}-{int(pop_size)}"
        stats.setdefault(dict_key, []).append(Result(formula_name,
                                                     int(run_no),
                                                     int(clause_cnt),
                                                     int(sat_clause_cnt),
                                                     int(weight),
                                                     int(last_improvement),
                                                     int(pop_size),
                                                     actual_max_weights.get(formula_name, 1),
                                                     ))

stats

{'218-600': [Result(formula_name='wuf50-01.mwcnf', run_no=1, clause_cnt=218, sat_clause_cnt=217, weight=10074, last_improvement=68, pop_size=600, max_weight=4190),
  Result(formula_name='wuf50-01.mwcnf', run_no=9, clause_cnt=218, sat_clause_cnt=218, weight=4190, last_improvement=157, pop_size=600, max_weight=4190),
  Result(formula_name='wuf50-02.mwcnf', run_no=1, clause_cnt=218, sat_clause_cnt=217, weight=13957, last_improvement=63, pop_size=600, max_weight=1469),
  Result(formula_name='wuf50-01.mwcnf', run_no=5, clause_cnt=218, sat_clause_cnt=218, weight=4190, last_improvement=227, pop_size=600, max_weight=4190),
  Result(formula_name='wuf50-01.mwcnf', run_no=0, clause_cnt=218, sat_clause_cnt=218, weight=4190, last_improvement=136, pop_size=600, max_weight=4190),
  Result(formula_name='wuf50-02.mwcnf', run_no=0, clause_cnt=218, sat_clause_cnt=217, weight=12036, last_improvement=71, pop_size=600, max_weight=1469),
  Result(formula_name='wuf50-01.mwcnf', run_no=8, clause_cnt=218, sat_c

In [15]:
from copy import deepcopy
from numpy import mean

filtered_stats = deepcopy(stats)
for key, result_list in stats.items():
    if result_list[0].clause_cnt == 430:
        filtered_stats.pop(key)

averages = []
for key, result_list in filtered_stats.items():
    averages.append(
        (mean([result.get_int() for result in result_list]),
         mean([result.get_penalized_last_improvement() for result in result_list]),
         mean([result.last_improvement for result in result_list]),
         key)
    )

averages.sort(reverse=True)
print("\n".join([str(avg) for avg in averages]))
print("-------")
averages.sort(key=lambda x: x[1])
print("\n".join([str(avg) for avg in averages]))

# for succ_avg, ppz_avg, key in averages[:10]:
#     results = stats[key]
#     for result in results:
#         file_name = f"out/{result.formula_name}-{result.run_no}-{result.tour_size}-{result.cross}-{float(result.mut)}"
#         debug_stream = StringIO()
#         with open(file_name, "rt") as f:
#             debug_stream.writelines(f.readlines())

#         print_plot(debug_stream=debug_stream,
#                    title=file_name,
#                    out_file=Path(file_name + ".png"))

(0.4507048697506525, 494.14, 137.38, '218-600')
-------
(0.4507048697506525, 494.14, 137.38, '218-600')


In [16]:
selected_hyperparameters = {
    "population_size": 100,
    "number_of_generations": 500,
    "selection": TournamentSelection(1.5),
    "crossover_probability": 0.9,
    "mutation_rate": 0.01,
    "elitism": 1
}

instances = [
    "wuf20-91/wuf20-91-Q/wuf20-01.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-02.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-03.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-04.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-05.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-06.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-07.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-08.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-09.mwcnf",
]

solved = 0
for instance in instances:
    for i in range(10):
        debug_stream = StringIO()
        formula_path = Path(instance)
        formula = parse_mwcnf(formula_path)
        solution, actual_generation_cnt, _ = genetic_algorithm(MwcnfGenerator(formula),
                                                            debug_stream=debug_stream,
                                                            **selected_hyperparameters)
        print_plot(debug_stream,
                   title=f"{formula_path.name}")

        clause_cnt = formula.clause_cnt
        sat_clause_cnt = -satisfied_clause_count(formula, solution.config)
        weight = -weights(formula, solution.config)
        print(f"{clause_cnt=}, {sat_clause_cnt=}, {weight=}")
        solved += clause_cnt == sat_clause_cnt

solved

NameError: name 'print_plot' is not defined